In [ ]:
import os
os.environ['WANDB_DISABLED']="true"

# Install and load libraries

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib rouge_score evaluate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-3b1aeb26-e6eb-ebe6-dfa3-09bd3a8ab2cc)


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
import torch

from datasets import Dataset, load_metric

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# set up the model with Lora

Fine-tune version of led-large-16384 - Allenai's Longformer Encoder-Decoder (LED). - https://huggingface.co/allenai/led-large-16384-arxiv - (https://https://arxiv.org/pdf/2004.05150)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb

model_name = "allenai/led-large-16384"
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    load_in_8bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="Seq2Seq" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 461374464 || trainable%: 0.34090833427660183


In [ ]:
!nvidia-smi

Wed Jul 17 03:28:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              50W / 400W |   1029MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Load data

In [ ]:
# Paths to the files
path_train_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_KeyBert/train_Short_Keybert.csv'
path_test_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_KeyBert/test_Short_Keybert.csv'
path_val_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_KeyBert/val_Short_Keybert.csv'

# Load the datasets
train_short = pd.read_csv(path_train_short)
test_short = pd.read_csv(path_test_short)
val_short = pd.read_csv(path_val_short)

# Check the first few rows to ensure they're loaded correctly
print("short_dialogue_train_data:")
print(train_short.head())

# Check the shape
print("Shape of short_dialogue_train_data:")
print(train_short.shape)
print("Shape of short_dialogue_test_data:")
print(test_short.shape)
print("Shape of short_dialogue_val_data:")
print(val_short.shape)


short_dialogue_train_data:
  section_header                                       section_text  \
0          GENHX  The patient is a 75-year-old female who comes ...   
1      FAM/SOCHX         Significant for diabetes and hypertension.   
2  PASTMEDICALHX                  Significant for anxiety disorder.   
3          GENHX  The patient is a 77-year-old female who is una...   
4      FAM/SOCHX                                   Noncontributory.   

                                            dialogue  \
0  Doctor: Welcome to the clinic. I am Doctor Fra...   
1  Doctor: Does anyone else in your family suffer...   
2  Doctor: Have we gone over your survey results ...   
3  Guest_clinician: How old is the patient? Docto...   
4  Doctor: Do you have a known- Patient: Drug all...   

                                   dialogue_keywords  
0  ['doctor did dizzy', 'stroke patient morning',...  
1  ['relatives actually doctor', 'family suffer h...  
2  ['doctor yes anxiety', 'survey results vi

In [ ]:
train_short.head(1)

,section_header,section_text,dialogue,dialogue_keywords
0,GENHX,The patient is a 75-year-old female who comes ...,Doctor: Welcome to the clinic. I am Doctor Fra...,"['doctor did dizzy', 'stroke patient morning',..."


In [ ]:
import ast

def convert_to_string(list_str):
    try:
        # Safely evaluate the string as a list
        actual_list = ast.literal_eval(list_str)
        # Join the list elements into a single string separated by ", "
        return ", ".join(actual_list)
    except Exception as e:
        # Print the error and problematic string
        print(f"Error: {e} with string: {list_str}")
        return list_str  # Return the original string if there's an error

# Apply the function to convert each list to a string
for df in [train_short, test_short, val_short]:
    df['dialogue_keywords'] = df['dialogue_keywords'].apply(convert_to_string)


In [ ]:
train_short.head(1)

,section_header,section_text,dialogue,dialogue_keywords
0,GENHX,The patient is a 75-year-old female who comes ...,Doctor: Welcome to the clinic. I am Doctor Fra...,"doctor did dizzy, stroke patient morning, numb..."


In [ ]:
# Load your datasets
train_short = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_KeyBert/train_Short_Keybert.csv')

# Tokenization function to count tokens
def count_tokens(text, tokenizer):
    return len(tokenizer.encode(text))

# Calculate dialogue statistics
dialogue_lengths = train_short['dialogue'].apply(lambda x: count_tokens(x, tokenizer))
dialogue_stats = {
    'min_length': np.min(dialogue_lengths),
    'max_length': np.max(dialogue_lengths),
    'average_length': np.mean(dialogue_lengths),
    'median_length': np.median(dialogue_lengths),
    '95_percentile_length': np.percentile(dialogue_lengths, 95)
}

# Calculate summary statistics
summary_lengths = train_short['section_text'].apply(lambda x: count_tokens(x, tokenizer))
summary_stats = {
    'min_length': np.min(summary_lengths),
    'max_length': np.max(summary_lengths),
    'average_length': np.mean(summary_lengths),
    'median_length': np.median(summary_lengths),
    '95_percentile_length': np.percentile(summary_lengths, 95)
}

print("Dialogue Statistics: ", dialogue_stats)
print("Summary Statistics: ", summary_stats)


Dialogue Statistics:  {'min_length': 12, 'max_length': 2137, 'average_length': 141.2514705882353, 'median_length': 88.0, '95_percentile_length': 432.1999999999998}
Summary Statistics:  {'min_length': 4, 'max_length': 1403, 'average_length': 54.90294117647059, 'median_length': 23.0, '95_percentile_length': 201.14999999999986}


In [ ]:
import pandas as pd
import numpy as np

# Load your datasets
train_short = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_KeyBert/train_Short_Keybert.csv')

# Tokenization function to count tokens
def count_tokens(text, tokenizer):
    return len(tokenizer.encode(text))

# Apply token counting to each entry in dialogue and summary columns
train_short['dialogue_token_count'] = train_short['dialogue'].apply(lambda x: count_tokens(x, tokenizer))
train_short['summary_token_count'] = train_short['section_text'].apply(lambda x: count_tokens(x, tokenizer))

# Calculate total token counts
total_dialogue_tokens = train_short['dialogue_token_count'].sum()
total_summary_tokens = train_short['summary_token_count'].sum()

print(f"Total Dialogue Tokens: {total_dialogue_tokens}")
print(f"Total Summary Tokens: {total_summary_tokens}")


Total Dialogue Tokens: 192102
Total Summary Tokens: 74668



- Min dialogue length: 12
- Ave dialogue length: 141
- Max dialogue length: 2137
- 95% dialogue length: 432
- median dialogue length: 88
- Min summary length: 4
- Ave summary length: 51
- Max summary length: 1403
- 95% summary length: 201
- Median summary length: 23

In [ ]:
train_short = Dataset.from_pandas(train_short)
test_short = Dataset.from_pandas(test_short)
val_short = Dataset.from_pandas(val_short)


# Fine-tuning the model

## Tokenization

In [ ]:
# tokenize
max_input_length = 2048
max_output_length = 1024
def process_data_to_model_inputs(batch):
    start_prompt = (
    "Summarize the following patient-doctor. "
    "Include all medically relevant information, including family history, "
    "diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. "
    "Use the following medical and chemical entities extracted from the dialogue to help summarization, but do not overly use them. "
    "Entities:\n\n")

    # Define the end of the prompt to indicate where the summary should go
    end_prompt = "\n\nSummary: "

    # Construct the full prompt for each dialogue in the batch, using the corresponding entities
    prompt = [start_prompt + entities + "\n\n" + 'Dialogue: \n\n' + dialogue + end_prompt
               for entities, dialogue in zip(batch['dialogue_keywords'], batch['dialogue'])]
    # tokenize the inputs and labels
    inputs = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["section_text"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
# map the function
batch_size = 2
train_short = train_short.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "section_header", "section_text","dialogue","dialogue_keywords"],
)

Map:   0%|          | 0/1360 [00:00<?, ? examples/s]

In [ ]:
test_short = test_short.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "section_header", "section_text","dialogue","dialogue_keywords"],
)

Map:   0%|          | 0/222 [00:00<?, ? examples/s]

In [ ]:
val_short = val_short.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=[ "section_header", "section_text","dialogue","dialogue_keywords"],
)

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

In [ ]:
from random import sample
# check the shape after tokenization
sample_input_id = test_short['input_ids'][0:1]
sample_label = test_short['labels'][0:1]
sample_attention_mask = test_short['attention_mask'][0:1]
sample_global_attention_mask = test_short['global_attention_mask'][0:1]

print("tokenized_train_exp: ", test_short)
print("\nsample_input_id: ", len(sample_input_id[0]), sample_input_id)
print("\nsample_label: ", len(sample_label[0]), sample_label)
print("\nsample_attention_mask: ", len(sample_attention_mask[0]), sample_attention_mask)
print("\nsample_global_attention_mask: ", len(sample_global_attention_mask[0]), sample_global_attention_mask)


tokenized_train_exp:  Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 222
})

sample_input_id:  2048 [[0, 38182, 3916, 2072, 5, 511, 3186, 12, 44914, 4, 39682, 70, 25749, 4249, 335, 6, 217, 284, 750, 6, 9726, 6, 375, 1131, 8, 15535, 750, 6, 13998, 18391, 6, 6348, 775, 6, 8, 684, 26331, 4, 7627, 5, 511, 1131, 8, 4747, 8866, 27380, 31, 5, 6054, 7, 244, 39186, 1938, 6, 53, 109, 45, 14860, 304, 106, 4, 9860, 2192, 35, 50118, 50118, 23846, 20280, 4910, 1215, 12368, 6, 972, 1623, 6, 546, 2536, 474, 6, 3299, 4664, 101, 6, 7154, 4076, 592, 50118, 50118, 48201, 35, 1437, 50118, 50118, 41152, 35, 12289, 6, 89, 4, 27690, 35, 12289, 4, 22809, 1215, 12368, 35, 12289, 328, 85, 16, 2579, 7, 972, 47, 4, 12521, 35, 85, 16, 2579, 7, 972, 47, 25, 157, 4, 370, 32, 5, 1623, 6, 4577, 116, 22809, 1215, 12368, 35, 3216, 4, 38, 524, 5, 1623, 4, 12521, 35, 1336, 251, 33, 47, 80, 57, 2997, 116, 22809, 1215, 12368, 35, 166, 33, 57, 2997, 13, 24503, 799, 10

In [ ]:
decode_text = tokenizer.decode(sample_input_id[0], skip_special_tokens=True)
decode_text

"Summarize the following patient-doctor. Include all medically relevant information, including family history, diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. Use the following medical and chemical entities extracted from the dialogue to help summarization, but do not overly use them. Entities:\n\npatient hi guest_family, meet husband, looking mental health, doctor recommendations like, rarely drink social\n\nDialogue: \n\nDoctor: Hi, there. Patient: Hi. Guest_family: Hi! It is nice to meet you. Doctor: It is nice to meet you as well. You are the husband, correct? Guest_family: Yes. I am the husband. Doctor: How long have you two been married? Guest_family: We have been married for forty eight years. Doctor: Do you live in the same household? Patient: Yes. Doctor: Do you have a primary care physician? Patient: No, I am currently looking for someone. I have also been looking for someone to see for my mental health issues. Doctor: I have a f

## Training

In [ ]:

import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import DataCollatorForSeq2Seq

# ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.optim import AdamW

# Define custom optimizer
optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)

# Define the output directory
#repository_id = "./output_directory"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=6,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    #logging & evaluation strategies
    #logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_short,
    eval_dataset=val_short,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),  # Pass the custom optimizer
)

Rouge SCores and len Gen are normailzed: it mean that what showned in the code below must be divided for 100.

In [ ]:
# Start training
with torch.cuda.amp.autocast():
    trainer.train()


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.869747,0.000000,0.000000,0.000000,0.000000,20.000000
2,No log,2.617629,0.000000,0.000000,0.000000,0.000000,20.000000
3,2.707600,2.584114,0.000000,0.000000,0.000000,0.000000,20.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1253: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1253: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 

In [ ]:
!nvidia-smi

Tue Jul 16 00:09:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0              71W / 400W |  11221MiB / 40960MiB |      1%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# save the model
trainer.save_model("./led_short_dialogue_MTS_keybert_unique_ner")
tokenizer.save_pretrained("./led_short_dialogue_MTS_keybert_unique_ner")

('./led_short_dialogue_MTS_keybert_unique_ner/tokenizer_config.json',
 './led_short_dialogue_MTS_keybert_unique_ner/special_tokens_map.json',
 './led_short_dialogue_MTS_keybert_unique_ner/vocab.json',
 './led_short_dialogue_MTS_keybert_unique_ner/merges.txt',
 './led_short_dialogue_MTS_keybert_unique_ner/added_tokens.json',
 './led_short_dialogue_MTS_keybert_unique_ner/tokenizer.json')

In [ ]:
# push to huggingface
from huggingface_hub import HfApi, HfFolder

model_name = "agnesem/led_short_dialogue_MTS_keybert_unique_ner"
api = HfApi()

# Create the repository if it doesn't exist
api.create_repo(repo_id=model_name, repo_type="model", exist_ok=True)

# Upload the model files
api.upload_folder(
    folder_path="./led_short_dialogue_MTS_keybert_unique_ner",
    repo_id=model_name,
    repo_type="model"
)

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/agnesem/led_short_dialogue_MTS_keybert_unique_ner/commit/9df0275c27b6b8665257383d7c688de42200fed0', commit_message='Upload folder using huggingface_hub', commit_description='', oid='9df0275c27b6b8665257383d7c688de42200fed0', pr_url=None, pr_revision=None, pr_num=None)

# Model evaluation

## Load the model with quantization




In [ ]:
!pip install -q accelerate
!pip install -q bitsandbytes

In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "agnesem/led_short_dialogue_MTS_keybert_unique_ner"

In [ ]:
from transformers import BitsAndBytesConfig


nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=nf4_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
# Paths to the filespath_test_short
path_test_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_KeyBert/test_Short_Keybert.csv'

# Load the datasets

test_short = pd.read_csv(path_test_short)


# Check the shape

print("Shape of short_dialogue_test_data:")
print(test_short.shape)


Shape of short_dialogue_test_data:
(222, 4)


In [ ]:
test_short.head(1)

,section_header,section_text,dialogue,dialogue_keywords
0,FAM/SOCHX,The patient lives with her husband of 48 years...,"Doctor: Hi, there. Patient: Hi. Guest_family: ...","['patient hi guest_family', 'meet husband', 'l..."


In [ ]:
import pandas as pd
import ast

# Define your path and load the data
path_test_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_KeyBert/test_Short_Keybert.csv'
test_short = pd.read_csv(path_test_short)

# Check data type
print("Data type of 'dialogue_keywords':", type(test_short['dialogue_keywords'].iloc[0]))

# Define a function to safely evaluate the string as a list and then convert it to a string
def convert_to_string(list_str):
    try:
        # Safely evaluate the string as a list
        actual_list = ast.literal_eval(list_str)
        # Join the list elements into a single string separated by ", "
        return ", ".join(actual_list)
    except Exception as e:
        # Print the error and problematic string
        print(f"Error: {e} with string: {list_str}")
        return list_str  # Return the original string if there's an error

# Apply the function to convert each list to a string
test_short['dialogue_keywords'] = test_short['dialogue_keywords'].apply(convert_to_string)

# Check results
print(test_short[['dialogue_keywords']].head())


Data type of 'dialogue_keywords': <class 'str'>
                                   dialogue_keywords
0  patient hi guest_family, meet husband, looking...
1  patient doctor energy, cancer past, sudden wei...
2  doctor hi welcome, history patient, doctor pri...
3  doctor taking adderall, man mother patient, pr...
4  pregnancy test doctor, menstrual period patien...


In [ ]:
test_short.head(1)

,section_header,section_text,dialogue,dialogue_keywords
0,FAM/SOCHX,The patient lives with her husband of 48 years...,"Doctor: Hi, there. Patient: Hi. Guest_family: ...","patient hi guest_family, meet husband, looking..."


In [ ]:
num_chunks = 3

# Split the DataFrame
dfs = np.array_split(test_short, num_chunks)

In [ ]:
# save dialogue and note to list for future Rough score
dialogue_lists = [subset["dialogue"].tolist() for subset in dfs]
note_lists = [subset["section_text"].tolist() for subset in dfs]

In [ ]:
datasets = []
for i in range(len(dfs)):
    dataset = Dataset.from_pandas(dfs[i])
    datasets.append(dataset)

In [ ]:
datasets

[Dataset({
     features: ['section_header', 'section_text', 'dialogue', 'dialogue_keywords'],
     num_rows: 74
 }),
 Dataset({
     features: ['section_header', 'section_text', 'dialogue', 'dialogue_keywords'],
     num_rows: 74
 }),
 Dataset({
     features: ['section_header', 'section_text', 'dialogue', 'dialogue_keywords'],
     num_rows: 74
 })]

## Tokenization

In [ ]:
# tokenize
max_input_length = 2048
max_output_length = 1024
def process_data_to_model_inputs(batch):
    start_prompt = (
    "Summarize the following patient-doctor. "
    "Include all medically relevant information, including family history, "
    "diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. "
    "Use the following medical and chemical entities extracted from the dialogue to help summarization, but do not overly use them. "
    "Entities:\n\n")

    # Define the end of the prompt to indicate where the summary should go
    end_prompt = "\n\nSummary: "

    # Construct the full prompt for each dialogue in the batch, using the corresponding entities
    prompt = [start_prompt + entities + "\n\n" + 'Dialogue: \n\n' + dialogue + end_prompt
               for entities, dialogue in zip(batch['dialogue_keywords'], batch['dialogue'])]
    # tokenize the inputs and labels
    inputs = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["section_text"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
batch_size = 2
for i in range(3):
    processed_dataset = datasets[i].map(
        process_data_to_model_inputs,
        batched=True,
        batch_size=batch_size,
        remove_columns=["section_header", "section_text","dialogue","dialogue_keywords"],
    )
    # Create a variable name dynamically
    globals()[f'dataset_{i}'] = processed_dataset

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

In [ ]:
dataset_0

Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 74
})

In [ ]:
## check the tokenization
sample_input_id = dataset_0['input_ids'][0:1]

In [ ]:
# check the prompt:
decoded_text = tokenizer.decode(sample_input_id[0], skip_special_tokens=True)
decoded_text

"Summarize the following patient-doctor. Include all medically relevant information, including family history, diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. Use the following medical and chemical entities extracted from the dialogue to help summarization, but do not overly use them. Entities:\n\npatient hi guest_family, meet husband, looking mental health, doctor recommendations like, rarely drink social\n\nDialogue: \n\nDoctor: Hi, there. Patient: Hi. Guest_family: Hi! It is nice to meet you. Doctor: It is nice to meet you as well. You are the husband, correct? Guest_family: Yes. I am the husband. Doctor: How long have you two been married? Guest_family: We have been married for forty eight years. Doctor: Do you live in the same household? Patient: Yes. Doctor: Do you have a primary care physician? Patient: No, I am currently looking for someone. I have also been looking for someone to see for my mental health issues. Doctor: I have a f

## Prediction

subset_0

In [ ]:
def model_generation(model, tokenizer, dataset):
    predictions = []

    for example in dataset:
        input_ids = example["input_ids"]
        attention_mask = example["attention_mask"]
        global_attention_mask = example["global_attention_mask"]
        reference_ids = example["labels"]

        # Decode input_ids
        input_text = tokenizer.decode(input_ids, skip_special_tokens=True)

        # Generate summary
        outputs = model.generate(input_ids=torch.tensor([input_ids]).to(device),
                                 attention_mask=torch.tensor([attention_mask]).to(device),
                                 global_attention_mask=torch.tensor([global_attention_mask]).to(device),
                                 num_beams= 4,
                                 min_length=100,
                                 max_length=512,
                                 length_penalty=2,
                                 no_repeat_ngram_size=3)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append to lists for ROUGE calculation
        predictions.append(generated_text)
    return predictions

In [ ]:
with torch.cuda.amp.autocast():
  predictions_sub_0 = model_generation(model, tokenizer, dataset_0)

In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(predictions_sub_0, note_lists[0]):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1: 0.17177868218106632
Average ROUGE-2: 0.07199219153700477
Average ROUGE-L: 0.10897600735930564


In [ ]:
data = {
    "Dialogue": dialogue_lists[0],
    "Reference": note_lists[0],
    "Prediction": predictions_sub_0,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/w266_Project Ideas/Results/Models/Keybert'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"led_large_lora_keybert_short_unique_predictions_sub_0_q.csv"), index=False)

subset_1

In [ ]:
with torch.cuda.amp.autocast():
  predictions_sub_1 = model_generation(model, tokenizer, dataset_1)
  from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(predictions_sub_1, note_lists[1]):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

data = {
    "Dialogue": dialogue_lists[1],
    "Reference": note_lists[1],
    "Prediction": predictions_sub_1,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Keybert'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"led_large_lora_keybert_short_unique_predictions_sub_1.csv"), index=False)

Average ROUGE-1: 0.17700831412872636
Average ROUGE-2: 0.0693495314114643
Average ROUGE-L: 0.11398653573021011


subset_2

In [ ]:
with torch.cuda.amp.autocast():
  predictions_sub_2 = model_generation(model, tokenizer, dataset_2)
  from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(predictions_sub_2, note_lists[2]):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

data = {
    "Dialogue": dialogue_lists[2],
    "Reference": note_lists[2],
    "Prediction": predictions_sub_2,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Keybert'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"led_large_lora_keybert_short_unique_predictions_sub_2.csv"), index=False)

Average ROUGE-1: 0.1707061305280737
Average ROUGE-2: 0.06524401507084747
Average ROUGE-L: 0.10681021287615461


Combine and evaluate overall

In [ ]:
# combine all the predictions:
predictions = []
for i in range(3):
  var_name = f"predictions_sub_{i}"
  predictions.extend(globals()[var_name])

In [ ]:
# Paths to the files
path_test_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_KeyBert/test_Short_Keybert.csv'

# Load the datasets

test_short = pd.read_csv(path_test_short)

# Check the shape

print("Shape of short_dialogue_test_data:")
print(test_short.shape)

Shape of short_dialogue_test_data:
(222, 4)


In [ ]:
dialogues = test_short["dialogue"].tolist()
reference = test_short["section_text"].tolist()

In [ ]:
data = {
    "Dialogue": dialogues,
    "Reference": reference,
    "Prediction": predictions,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Keybert'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"led_lora_keybert_short_unique_predictions.csv"), index=False)

In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(predictions,reference):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1: 0.17316437561262216
Average ROUGE-2: 0.06886191267310549
Average ROUGE-L: 0.10992425198855678
